In [75]:
from Bio.Entrez import efetch, read
from Bio import Entrez
import pandas as pd
from tqdm.notebook import tqdm
import gender_guesser.detector as gender

Entrez.email = "singhmohit.1889@gmail.com"
Entrez.api_key = "1c22910bfa9f49693b451d3dfec15c1dbb09"
# pip install biopython
# pip install gender-guesser

d = gender.Detector()
def get_gender(name):
    first_name = name.split()[0]
    gender = d.get_gender(first_name)
    return gender

# pmids = pd.read_csv('pmid.txt')['pmid']

pmids = [24459728]
# pmids = [17139226,
# 23753526,
# 21782045,
# 21626934,
# 23954563,
# 14550805,
# 15187116,
# 12808441,
# 16210359,
# 14581154,
# 22253368,
# 18480896,
# 25291577,
# 21987784,
# 23441200,
# 15207317]

# df1 = pd.DataFrame([],columns=['PMID','Title','Authors'])
def fetch_abstract(pmid):
    try:
        
        handle = efetch(db='pubmed', id=pmid, retmode='xml')
        xml_data = read(handle)

        auth = [i['ForeName'] + ' '+ i['LastName'] for i in xml_data['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList'] if 'ForeName' in i and 'LastName' in i]

        title = xml_data['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
        
#         df['male'] = df['Gender'].progress_apply(lambda x: x.count('male'))
#         df['female'] = df['Gender'].progress_apply(lambda x: x.count('female'))
#         df['unknown'] = df['Gender'].progress_apply(lambda x: x.count('unknown'))

        df.loc[len(df)] = [pmid, title, auth]
    except Exception as e:
        print(e)

In [76]:
len(pmids)

1

In [77]:
for pid in tqdm(pmids):
    fetch_abstract(pid)

  0%|          | 0/1 [00:00<?, ?it/s]

In [79]:
df['Gender'] = df['Authors'].apply(lambda author_names: [get_gender(name) for name in author_names])

In [80]:
df 

,PMID,Title,Authors,Gender
0,19456161,"Macrocyclic design strategies for small, stabl...","[Felix Freire, Samuel H Gellman]","[male, male]"
1,19580264,An alpha/beta-peptide helix bundle with a pure...,"[Michael W Giuliano, W Seth Horne, Samuel H Ge...","[male, unknown, male]"
2,19644993,In situ monitoring of backbone thioester excha...,"[William C Pomerantz, Erik B Hadley, Charles G...","[male, male, male, male]"
3,19967682,Detection and analysis of chimeric tertiary st...,"[Joshua L Price, Erik B Hadley, Jay D Steinkru...","[male, male, male, male]"
4,20465308,Side-chain pairing preferences in the parallel...,"[Jay D Steinkruger, Derek N Woolfson, Samuel H...","[male, male, male]"
...,...,...,...,...
32666,25291577,Serological responses to an avian influenza A/...,"[Mark J Mulligan, David I Bernstein, Patricia ...","[male, male, female, male, male, female, femal..."
32667,21987784,Hydroxyurea induces de novo copy number varian...,"[Martin F Arlt, Alev Cagla Ozdemir, Shanda R B...","[male, female, female, male, male]"
32668,23441200,Diaphanous homolog 3 (Diap3) overexpression ca...,"[Cynthia J Schoen, Margit Burmeister, Marci M ...","[female, female, female]"
32669,15207317,M2 muscarinic receptors in pontine reticular f...,"[C G Coleman, R Lydic, H A Baghdoyan]","[unknown, unknown, unknown]"


In [22]:
from concurrent.futures import ThreadPoolExecutor
from Bio.Entrez import efetch, read
from Bio import Entrez
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
import gender_guesser.detector as gender

# pip install metapub
# pip install gender-guesser

d = gender.Detector()
def get_gender(name):
    first_name = name.split()[0]
    gender = d.get_gender(first_name)
    return gender


Entrez.email = "singhmohit.1889@gmail.com"
Entrez.api_key = "1c22910bfa9f49693b451d3dfec15c1dbb09"

pmids = pd.read_csv('pmid.txt')['pmid']
    
# pmids = [17139226,
# 23753526,
# 21782045,
# 21626934,
# 23954563,
# 14550805,
# 15187116,
# 12808441,
# 16210359,
# 14581154,
# 22253368,
# 18480896,
# 25291577,
# 21987784,
# 23441200,
# 15207317]

df = pd.DataFrame([], columns=['PMID', 'Title', 'Authors'])

def fetch_abstract(pmid):
    try:
        handle = efetch(db='pubmed', id=pmid, retmode='xml')
        xml_data = read(handle)

        auth = [i['ForeName'] + ' '+ i['LastName'] for i in xml_data['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList'] if 'ForeName' in i and 'LastName' in i]
        title = xml_data['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
        
        return pmid, title, auth
    except Exception as e:
        print(f"pmid-{pmid}",e)
        return None

def process_pmids(pmids):
    results = []
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(fetch_abstract, pmid) for pmid in pmids]
        for future in tqdm(futures, total=len(futures), desc="Fetching Abstracts"):
            result = future.result()
            if result:
                results.append(result)
    return results

results = process_pmids(pmids)

for result in results:
    df.loc[len(df)] = result

Fetching Abstracts:   0%|          | 0/32671 [00:00<?, ?it/s]

pmid-17139226 HTTP Error 429: Too Many Requests
pmid-23753526 HTTP Error 429: Too Many Requests
pmid-21782045 list index out of range
pmid-21626934 HTTP Error 429: Too Many Requests
pmid-23954563 HTTP Error 429: Too Many Requests
pmid-14550805 Failed to find tag 'pubmed' in the DTD. To skip all tags that are not represented in the DTD, please call Bio.Entrez.read or Bio.Entrez.parse with validate=False.
pmid-15187116 Failed to find tag 'pubmed' in the DTD. To skip all tags that are not represented in the DTD, please call Bio.Entrez.read or Bio.Entrez.parse with validate=False.
pmid-12808441 Failed to find tag 'pubmed' in the DTD. To skip all tags that are not represented in the DTD, please call Bio.Entrez.read or Bio.Entrez.parse with validate=False.
pmid-16210359 HTTP Error 429: Too Many Requests
pmid-14581154 HTTP Error 429: Too Many Requests
pmid-22253368 HTTP Error 429: Too Many Requests
pmid-18480896 HTTP Error 429: Too Many Requests
pmid-25291577 HTTP Error 429: Too Many Requests

In [40]:
df['Gender'] = df['Authors'].progress_apply(lambda author_names: [get_gender(name) for name in author_names])

  0%|          | 0/32670 [00:00<?, ?it/s]

In [34]:
# 21782045 

In [81]:
df['male'] = df['Gender'].progress_apply(lambda x: x.count('male'))
df['female'] = df['Gender'].progress_apply(lambda x: x.count('female'))
df['unknown'] = df['Gender'].progress_apply(lambda x: x.count('unknown'))

  0%|          | 0/32671 [00:00<?, ?it/s]

  0%|          | 0/32671 [00:00<?, ?it/s]

  0%|          | 0/32671 [00:00<?, ?it/s]

In [82]:
df

,PMID,Title,Authors,Gender,male,female,unknown
0,19456161,"Macrocyclic design strategies for small, stabl...","[Felix Freire, Samuel H Gellman]","[male, male]",2,0,0
1,19580264,An alpha/beta-peptide helix bundle with a pure...,"[Michael W Giuliano, W Seth Horne, Samuel H Ge...","[male, unknown, male]",2,0,1
2,19644993,In situ monitoring of backbone thioester excha...,"[William C Pomerantz, Erik B Hadley, Charles G...","[male, male, male, male]",4,0,0
3,19967682,Detection and analysis of chimeric tertiary st...,"[Joshua L Price, Erik B Hadley, Jay D Steinkru...","[male, male, male, male]",4,0,0
4,20465308,Side-chain pairing preferences in the parallel...,"[Jay D Steinkruger, Derek N Woolfson, Samuel H...","[male, male, male]",3,0,0
...,...,...,...,...,...,...,...
32666,25291577,Serological responses to an avian influenza A/...,"[Mark J Mulligan, David I Bernstein, Patricia ...","[male, male, female, male, male, female, femal...",6,7,1
32667,21987784,Hydroxyurea induces de novo copy number varian...,"[Martin F Arlt, Alev Cagla Ozdemir, Shanda R B...","[male, female, female, male, male]",3,2,0
32668,23441200,Diaphanous homolog 3 (Diap3) overexpression ca...,"[Cynthia J Schoen, Margit Burmeister, Marci M ...","[female, female, female]",0,3,0
32669,15207317,M2 muscarinic receptors in pontine reticular f...,"[C G Coleman, R Lydic, H A Baghdoyan]","[unknown, unknown, unknown]",0,0,3


In [86]:
df.to_csv('PMID.csv',index=False)

In [85]:
df

,PMID,Title,Authors,Gender,male,female,unknown
0,19456161,"Macrocyclic design strategies for small, stabl...","[Felix Freire, Samuel H Gellman]","[male, male]",2,0,0
1,19580264,An alpha/beta-peptide helix bundle with a pure...,"[Michael W Giuliano, W Seth Horne, Samuel H Ge...","[male, unknown, male]",2,0,1
2,19644993,In situ monitoring of backbone thioester excha...,"[William C Pomerantz, Erik B Hadley, Charles G...","[male, male, male, male]",4,0,0
3,19967682,Detection and analysis of chimeric tertiary st...,"[Joshua L Price, Erik B Hadley, Jay D Steinkru...","[male, male, male, male]",4,0,0
4,20465308,Side-chain pairing preferences in the parallel...,"[Jay D Steinkruger, Derek N Woolfson, Samuel H...","[male, male, male]",3,0,0
...,...,...,...,...,...,...,...
32666,25291577,Serological responses to an avian influenza A/...,"[Mark J Mulligan, David I Bernstein, Patricia ...","[male, male, female, male, male, female, femal...",6,7,1
32667,21987784,Hydroxyurea induces de novo copy number varian...,"[Martin F Arlt, Alev Cagla Ozdemir, Shanda R B...","[male, female, female, male, male]",3,2,0
32668,23441200,Diaphanous homolog 3 (Diap3) overexpression ca...,"[Cynthia J Schoen, Margit Burmeister, Marci M ...","[female, female, female]",0,3,0
32669,15207317,M2 muscarinic receptors in pontine reticular f...,"[C G Coleman, R Lydic, H A Baghdoyan]","[unknown, unknown, unknown]",0,0,3


In [53]:
pmids = pd.read_csv('pmid.txt')['pmid']

In [73]:
df.drop(columns=['Gender'],axis=1,inplace=True)

In [74]:
df

,PMID,Title,Authors
0,19456161,"Macrocyclic design strategies for small, stabl...","[Felix Freire, Samuel H Gellman]"
1,19580264,An alpha/beta-peptide helix bundle with a pure...,"[Michael W Giuliano, W Seth Horne, Samuel H Ge..."
2,19644993,In situ monitoring of backbone thioester excha...,"[William C Pomerantz, Erik B Hadley, Charles G..."
3,19967682,Detection and analysis of chimeric tertiary st...,"[Joshua L Price, Erik B Hadley, Jay D Steinkru..."
4,20465308,Side-chain pairing preferences in the parallel...,"[Jay D Steinkruger, Derek N Woolfson, Samuel H..."
...,...,...,...
32665,18480896,Determination of glucose using a coupled-enzym...,"[Hisham S M Abd-Rabboh, Mark E Meyerhoff]"
32666,25291577,Serological responses to an avian influenza A/...,"[Mark J Mulligan, David I Bernstein, Patricia ..."
32667,21987784,Hydroxyurea induces de novo copy number varian...,"[Martin F Arlt, Alev Cagla Ozdemir, Shanda R B..."
32668,23441200,Diaphanous homolog 3 (Diap3) overexpression ca...,"[Cynthia J Schoen, Margit Burmeister, Marci M ..."


In [73]:
from Bio import Entrez
import pandas as pd
from tqdm.notebook import tqdm

Entrez.email = "singhmohit.1889@gmail.com"
Entrez.api_key = "1c22910bfa9f49693b451d3dfec15c1dbb09"

def fetch_paper_details(pmid):
    handle = Entrez.efetch(db="pubmed", id=pmid, rettype="xml", retmode="text")
    records = Entrez.read(handle)
    handle.close()
    paper_title = records['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
    authors = records['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList']
    paper_authors = [f"{author['ForeName']} {author['LastName']}" for author in authors]
    journal = records['PubmedArticle'][0]['MedlineCitation']['Article']['Journal']['Title']
    year = records['PubmedArticle'][0]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year']   
    return paper_title, paper_authors, journal, year

def fetch_citations(pmid):
    handle = Entrez.elink(dbfrom="pubmed", id=pmid, linkname="pubmed_pubmed_citedin")
    records = Entrez.read(handle)
    handle.close()
    
    citing_paper_titles = []
    citing_paper_references = []
    details = []
    if records[0]["LinkSetDb"]:
        for link in records[0]["LinkSetDb"][0]["Link"]:
            citing_pmid = link["Id"]
            handle = Entrez.efetch(db="pubmed", id=citing_pmid, rettype="xml", retmode="text")
            records = Entrez.read(handle)
            handle.close()

            paper_title = records['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
            references = records['PubmedArticle'][0]['PubmedData']['ReferenceList'][0]['Reference']
            reference_list = [reference['Citation'] for reference in references]
            details.append((paper_title,reference_list))
    
    return details

In [74]:
df = pd.read_csv("savedrecs imp csv.csv",encoding='latin-1')
PubmedId = df[~df['Pubmed Id'].isna()]['Pubmed Id'].values

In [75]:
PubmedId

array([37371829., 37020354., 36867683., 36167467., 35704995., 35536311.,
       35131437., 35214060., 34852217., 34910933., 34580453., 34516826.,
       34411542., 34103491., 34044428., 33469214., 33414215., 32783919.,
       32998157., 30933724., 30974332., 30819612., 30905578., 30024272.,
       30135514., 29733456., 29739835., 28985209., 28604701., 28151488.,
       28251988., 26264835., 26321183., 26121660., 25233993., 24211294.,
       22301437., 22334017., 21813458., 21890902., 22028224., 21876533.,
       21297615., 20457754., 18797453., 18368052., 17786850., 17709527.,
       16725117., 16111635., 15545925., 15367593., 15529017., 15153468.,
       15034009., 14729660., 15388987., 12391237., 11986215., 11821398.,
       11731008., 11295236., 11166368., 11035115., 10767559., 10455412.,
        9850143.,  9684800.,  9518670.,  9454902.,  9445265.,  8663495.,
        8669483.,  8566799.,  7546706.,  7890029.,  7925939.,  8202384.,
        7510679.,  8017832., 21573478.,  8391387., 

In [53]:
df = pd.DataFrame([],columns=['Paper_Title','Paper_Authors','Journal','Year','Citing_Paper_Title','Citing_Paper_References'])
for pmid in tqdm(PubmedId):
    paper_title, paper_authors, journal, year = fetch_paper_details(pmid)
    citing_papers = fetch_citations(pmid)
    for title, ref in citing_papers:
        df.loc[len(df)] = [paper_title, paper_authors, journal, year,title,ref]

29588180
22296518
21643549
